In [1]:
import tensorflow as tf

## Neural Network = Input Times Weight, add a bias, Activate

### Defining workflow
### input > weight > hidden layer1 > activation function > weights > hidden layer 2 > activation function > weights > output (!yeah)
### how close you are to the output, cost function, apply optimizer to minimize cost function, several to choose from, for example-- Gradient Descent, Adam Optimizer etc...
### Backpropagation to update weights in order to minimize cost function
### 1 Forward Propagation + 1 Backward Propagation. = 1 epoch

In [3]:
# importing the data
from tensorflow.examples.tutorials.mnist import input_data

Reading the dataset, one hot = True sets up one hot encoding to True. There is a good explanation for OHE here:  
 https://goo.gl/Eyrhzz

In [6]:
mnist = input_data.read_data_sets("/tmp/data/", one_hot=True)

Successfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz
Successfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Successfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Successfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [7]:
# Defining the model, this can be anything
nodes_hidden_layer_1 = 500
nodes_hidden_layer_2 = 600
nodes_hidden_layer_3 = 700
nodes_hidden_layer_4 = 100

In [8]:
#We have 10 classes for the classification here, from 0...9, therefore,
num_classes = 10

In [9]:
# We have to define the batch size, this means that it will do the whole computation in batches of 100,
# Basically taking 100 features at a time.
batch_size = 100

In [11]:
'''We have to define place holders for the input and output to run whole thing in session, 784 is 28*28 to 
define the shape of the image''' 
X = tf.placeholder(tf.float32, [None, 784])
Y = tf.placeholder(tf.float32)

In [24]:
'''Defining Neural Network
Weights and Biases are tensorflow variable. Biases are added to Input times Variable, 
workflow is basically first hidden layer input will be the same as our input. 
However, from the second layer, the output of first will become input of second and so on
and so forth. After last hidden layer, output we desire to have a class, so that is where num_classes 
comes into play
'''

def neural_net(Dataset_input):
    
    hidden_layer_1 = {'weights':tf.Variable(tf.random_normal([784, nodes_hidden_layer_1]))
                     , 'bias':tf.Variable(tf.random_normal([nodes_hidden_layer_1]))}
    
    hidden_layer_2 = {'weights':tf.Variable(tf.random_normal([nodes_hidden_layer_1, nodes_hidden_layer_2]))
                     , 'bias':tf.Variable(tf.random_normal([nodes_hidden_layer_2]))}
    
    hidden_layer_3 = {'weights':tf.Variable(tf.random_normal([nodes_hidden_layer_2, nodes_hidden_layer_3]))
                     , 'bias':tf.Variable(tf.random_normal([nodes_hidden_layer_3]))}
    
    hidden_layer_4 = {'weights':tf.Variable(tf.random_normal([nodes_hidden_layer_3, nodes_hidden_layer_4]))
                     , 'bias':tf.Variable(tf.random_normal([nodes_hidden_layer_4]))}
    
    output_layer = {'weights':tf.Variable(tf.random_normal([nodes_hidden_layer_4, num_classes]))
                     , 'bias':tf.Variable(tf.random_normal([num_classes]))}
    
    '''Defining Computation'''
    
    layer_1 = tf.add(tf.matmul(X,hidden_layer_1['weights']), hidden_layer_1['bias'])
    #Applying activation function
    layer_1 = tf.nn.relu(layer_1)
    
    layer_2 = tf.add(tf.matmul(layer_1,hidden_layer_2['weights']), hidden_layer_2['bias'])
    layer_2 = tf.nn.relu(layer_2)
    
    layer_3 = tf.add(tf.matmul(layer_2,hidden_layer_3['weights']), hidden_layer_3['bias'])
    layer_3 = tf.nn.relu(layer_3)
    
    layer_4 = tf.add(tf.matmul(layer_3,hidden_layer_4['weights']), hidden_layer_4['bias'])
    layer_4 = tf.nn.relu(layer_4)
    
    output = tf.add(tf.matmul(layer_4,output_layer['weights']), output_layer['bias'])
    
    return output

In [48]:
#Defining train Function

def train(X):
    
    pred = neural_net(X)
    
    cost_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = pred,labels=Y))
    '''Mininmizing cost function'''
    # Using Gradient Descent
    optimizer = tf.train.AdamOptimizer().minimize(cost_function)
    
    
    Num_Epochs = 3
    
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables()) # All it does is initialize the variables, weights and bias what we need to initialize
        
        for epoch in range(Num_Epochs):
            loss = 0
            for n in range(int(mnist.train.num_examples/batch_size)):
                epoch_x, epoch_y = mnist.train.next_batch(batch_size=batch_size)
                n, c = sess.run([optimizer, cost_function], feed_dict ={X:epoch_x,Y:epoch_y})
                loss +=c #adding cost 
            print('Epoch ', epoch, 'out of ', Num_Epochs, 'loss: ', loss)
        
        # tf.argmax ---> returns index of the maximum value, below line is just verifying that we are getting no more than 1 as output because it is one hot encoded
        check = tf.equal(tf.argmax(pred,1), tf.argmax(Y,1))
        # tf.cast just changes to the dtype
        accuracy = tf.reduce_mean(tf.cast(check, tf.float32))
        print('Accuracy: ', accuracy.eval({X:mnist.test.images, Y:mnist.test.labels}))

In [47]:
train(X)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch  0 out of  3 loss:  12503837.7043
Epoch  1 out of  3 loss:  2761520.9306
Epoch  2 out of  3 loss:  1450771.19055
Accuracy:  0.9254


# Yayyyy!! We did pretty well, we scored 92% accuracy on the test set, that's not bad at all.
